In [27]:

import numpy as np

game_field = np.array(
    [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
     [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1,  0,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
     [-1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1],
     [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
     [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

agent = ("something", 0, True, (5,5))
others = [("something_2", 0, True, (8,5)), ("something_3", 0, True, (2,5))]

game_state = {'field': game_field, 'self': agent, 'explosion_map': np.zeros((17,17)), 'bombs': [], 'others': others, 'coins': []}

In [28]:

MAX_WAITING_TIME = 2

## Game constants
ACTIONS              = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']
DIRECTIONS           = np.array([(0, -1), (1, 0), (0, 1), (-1, 0)])   # UP, RIGHT, DOWN, LEFT
DEFAULT_DISTANCE     = 1000
BOMB_COOLDOWN_TIME   = 7
COLS = ROWS          = 17
BLAST                = np.array([-3, -2, -1, 1, 2, 3])

## Calculate constant BOMB_MASK one time
BOMB_MASK = np.full((COLS, ROWS, COLS, ROWS), False)

x_inside = lambda x: x > 0 and x < COLS-1
y_inside = lambda y: y > 0 and y < ROWS-1

for x in range(1, COLS-1):
        for y in range(1, ROWS-1):
            if (x % 2 == 1 or y % 2 == 1):
                explosion_spots = [(x, y)]
                if x % 2 == 1:
                    explosion_spots += [(x, y + b) for b in BLAST  if y_inside(y + b)]
                if y % 2 == 1:
                    explosion_spots += [(x + b, y) for b in BLAST  if x_inside(x + b)]
                
                explosion_spots = tuple(np.array(explosion_spots).T)
                BOMB_MASK[(x, y)][explosion_spots] \
                                = True

In [29]:

def create_mask(positions, shape = (ROWS, COLS)):
    array = np.zeros(shape)
    if len(positions) > 0:
        indices        = tuple(np.array(positions).T)
        array[indices] = True
        return(array)
    else:
        return(array)


def build_free_spacetime_map(own_position, game_field, explosion_map, bombs, foe_map):
    free_spacetime = np.resize(np.logical_and(game_field == 0, foe_map == 0), (7, ROWS, COLS)) # exclude crates, walls and foes
    free_spacetime[1][np.nonzero(explosion_map)] = False # exclude present explosions
    free_spacetime[0][np.nonzero(explosion_map)] = False

    for ((x,y), bomb_timer) in bombs: 
        steps_until_explosion = bomb_timer + 1
        start                 = 1 if (x,y) == own_position else 0
        
        # Exclude bomb spots as long as bomb is present
        free_spacetime[start:steps_until_explosion, x, y] \
                              = np.zeros(steps_until_explosion - start)
        
        # include crates and foes destroyed by explosion 
        crates_or_foes_mask   = np.logical_or(game_field == 1, foe_map)
        what_gets_destroyed   = np.logical_and(BOMB_MASK[(x,y)], crates_or_foes_mask)
        block_until_destroyed = np.resize(what_gets_destroyed, (7 - steps_until_explosion, ROWS, COLS))
        free_spacetime[steps_until_explosion:][block_until_destroyed] \
                              = True 
        
        # exclude future explosions as long as present
        bomb_spread_mask      = np.resize(BOMB_MASK[(x,y)], (2, ROWS, COLS))
        free_spacetime[steps_until_explosion : steps_until_explosion+2][bomb_spread_mask] \
                              = False 

    return(free_spacetime)




def proximity_map (own_position, free_spacetime_map, explosion_map, bombs):
    """
    calculates three values for each tile of the game field:
    1. travel time aka. distance from own position
    2. if tile is reachable from own position or blocked
    3. in which directions one can initially go to reach that tile as quick as possible

    Arguments
    ---------
    own_position : tuple (x, y)
        with x and y being current coordinates coordinates of the agent 
        on the game field. Thus 0 < x < COLS-1, 0 < y < ROWS-1.
    ...

    Returns
    -------
    travel_time_map         : np.array, shape like game_field, dtype = int
        Reachable tiles have the value of the number of steps it takes to move to them 
        from own_position.
        Unreachable tiles have the value of DEFAULT_DISTANCE which is much higher than 
        any reachable time.
    reachable_map           : np.array, shape like game_field, dtype = bool
        A boolean mask of travel_time_map that labels reachable tiles as True and 
        unreachable ones as False.
    original_directions_map : np.array, shape = (COLS, ROWS, 5), dtype = bool
        A map of the game_field that holds a 5-element boolean array for every tile.
        Values of the tile's array correspond to the 5 directions UP, RIGHT, DOWN, LEFT, WAIT 
        which you might from own_position to reach the tile. Those direction which lead you 
        to reach the tile the fastest are marked True, the others False.
        For example, if you can reach a tile the fastest by either going UP or RIGHT at the step
        then its array will look like this [TRUE, TRUE, FALSE, FALSE, FALSE].
        This map will be important to quickly find the best direction towards coins, crates,
        opponents and more.
    """

    # Setup of initial values
    distance_time_map  = np.full((7, ROWS, COLS), DEFAULT_DISTANCE)
    direction_map = np.full((7, ROWS, COLS, 5), False) # UP, RIGHT, DOWN, LEFT, WAIT
    x_own, y_own = own_position

    distance_time_map[0, x_own, y_own] = 0
    direction_map[0, x_own, y_own][4] = free_spacetime_map[1, x_own, y_own]
    for i, step in enumerate([(0, -1), (1, 0), (0, 1), (-1, 0)]):
        x_next, y_next = np.array(step) + np.array(own_position)
        direction_map[1, x_next, y_next, i] = free_spacetime_map[1, x_next, y_next] # If neighbor is a free field in next step

    # Breadth first search for proximity values to all reachable spots
    frontier = [(0, x_own, y_own)]
    while len(frontier) > 0:
        t_current, x_current, y_current = frontier.pop(0)

        if not np.any(explosion_map) and bombs == []:
            waiting_time_limit = 1
        else: 
            currents_future = free_spacetime_map[min(t_current, 6):, x_current, y_current]
            waiting_time_limit = MAX_WAITING_TIME + 1 if np.all(currents_future) else min(MAX_WAITING_TIME + 1, np.argmin(currents_future))

        for waiting_time in range(waiting_time_limit):
            t_neighbor = t_current + waiting_time + 1

            for dir in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                x_neighbor, y_neighbor = np.array(dir) + np.array([x_current, y_current])
                neighbor = (min(t_neighbor, 6), x_neighbor, y_neighbor)
                
                # Update travel time to `neighbor` field
                if free_spacetime_map[neighbor]:
                    if distance_time_map[neighbor] > t_neighbor:
                        distance_time_map[neighbor] = t_neighbor
                        frontier.append((t_neighbor, x_neighbor, y_neighbor))
                    
                        # Update original direction for `neighbor` field
                        if t_neighbor > 1:
                            direction_map[neighbor] = direction_map[min(t_current, 6), x_current, y_current]
                            # print(f"1: {current}, {neighbor}")
                        
                    # Combine orginial directions if travel times are equal
                    elif distance_time_map[neighbor] == t_neighbor:
                        direction_map[neighbor] = np.logical_or(direction_map[neighbor], direction_map[min(t_current, 6), x_current, y_current])
                        # print(f"2: {current}, {neighbor}")

    shortest_distance_map = np.amin(distance_time_map, axis = 0)
    direction_map = np.take_along_axis(direction_map, np.argmin(distance_time_map, axis = 0).reshape(1, COLS, ROWS, 1), axis = 0).reshape(ROWS, COLS, 5)

    # Derivation of reachability_map
    reachability_map = shortest_distance_map != DEFAULT_DISTANCE

    return shortest_distance_map, reachability_map, direction_map


In [30]:
import numpy as np

# 0. Collect relevant game_state info
own_position      = game_state['self'][3]
can_place_bomb    = game_state['self'][2]
crate_map         = game_state['field']
collectable_coins = game_state['coins']
bombs             = game_state['bombs']
explosion_map     = game_state['explosion_map']
foes              = game_state['others']

neighbors     = own_position + DIRECTIONS
foe_positions = [foe[3] for foe in foes]
foe_map = create_mask(foe_positions)
foe_count = len(foe_positions)

# 1. Calculate proximity map
free_spacetime_map = build_free_spacetime_map(own_position, crate_map, explosion_map, bombs, foe_map)
distance_map, reachability_map, direction_wait_map \
                       = proximity_map(own_position, free_spacetime_map, explosion_map, bombs)
direction_map = direction_wait_map[:,:,:4]
#distance_map, reachability_map, direction_map = proximity_map(own_position, crate_map)


#####

bombing_is_dumb = False

if not bombing_is_dumb:
        no_future_explosion_mask = np.logical_not(BOMB_MASK[own_position])
        rescue_distances         = distance_map[reachability_map & no_future_explosion_mask] # improve by including explosions
        minimal_rescue_distance  = DEFAULT_DISTANCE if (rescue_distances.size == 0) else np.amin(rescue_distances) 
        # self.logger.debug(f'stf(): minimal rescue distance for own bombing: {minimal_rescue_distance}')
        if minimal_rescue_distance > 4:
            bombing_is_dumb = True
            # self.dumb_bombing_map[own_position] = bombing_is_dumb 

print(bombing_is_dumb)

True
